In [1]:
import pandas as pd
import numpy as np

In [ ]:
"""
Objectives:
1) Determine the number of students per school that have 0-4 'Yes's (matches based on value system as is)
Procedure:
<> Compile df that produces the 6 ordered choices per student by the value system
<> filter/iterate by school
"""

In [2]:
X = pd.read_csv('../Heatmaps/RCS_compiled.csv')
X.drop('Unnamed: 0', axis=1, inplace=True)
X.head()

,id,organization,last_name,first_name,email,phone,gender,ethnicity,grad_year,aptitude_fit,interest_fit,fit_rank
0,409512,Rocky Fork Middle School,Cardoso,Sofia,scardosoperez8225@student.rcschools.net-moved-...,615-900-7342,f,other,2023.0,Agriculture & Natural Resources,Human Services,3
1,409512,Rocky Fork Middle School,Cardoso,Sofia,scardosoperez8225@student.rcschools.net-moved-...,615-900-7342,f,other,2023.0,Finance,Arts & Media,2
2,409512,Rocky Fork Middle School,Cardoso,Sofia,scardosoperez8225@student.rcschools.net-moved-...,615-900-7342,f,other,2023.0,Engineering,Teaching,1
3,409514,Rocky Fork Middle School,Acevedo,Ubaldo,ubarajas5459@student.rcschools.net-moved-to-hs,615-904-6780,m,hispanic,2023.0,Agriculture & Natural Resources,Law & Public Safety,3
4,409514,Rocky Fork Middle School,Acevedo,Ubaldo,ubarajas5459@student.rcschools.net-moved-to-hs,615-904-6780,m,hispanic,2023.0,Architecture & Construction,Arts & Media,2


In [74]:
def get_choices_by_school(school = None):
    # filtering school choice
    if school:
        df = X[X.organization == school]
    else:
        df = X

    id_list = list(set(df.id))
    criteria = {
        'id':[],
        'name':[],
        'email':[],
        'First':[],
        'Second':[],
        'Third':[],
        'Fourth':[],
        'Fifth':[],
        'Sixth':[],
        'Enough Choices':[]
    }
    for id in id_list:
        Y = df.loc[df.id == id]
        criteria['id'].append(int(id))
        name = str(Y.first_name.values[0]) + ' ' + str(Y.last_name.values[0])
        criteria['name'].append(name)
        criteria['email'].append(list(Y.email)[0].split('-')[0])
        apt = Y[Y.fit_rank == 1]['aptitude_fit'].values[0]
        # creating a flag in the event that a student does not have 4 unique YS clusters from ranking system
        # First choice = 1st aptitude
        c1 = Y[Y.fit_rank == 1]['aptitude_fit'].values[0]
        criteria['First'].append(c1)
        # second choice = 1st interest
        c2 = Y[Y.fit_rank == 1]['interest_fit'].values[0]
        criteria['Second'].append(c2)
        # third choice = 2nd aptitude
        c3 = Y[Y.fit_rank == 2]['aptitude_fit'].values[0]
        criteria['Third'].append(c3)
        # fourth choice = 3rd aptitude
        c4 = Y[Y.fit_rank == 3]['aptitude_fit'].values[0]
        criteria['Fourth'].append(c4)
        # fifth choice = 2nd interest
        c5 = Y[Y.fit_rank == 2]['interest_fit'].values[0]
        criteria['Fifth'].append(c5)
        # sixth choice = 3rd interest
        c6 = Y[Y.fit_rank == 3]['interest_fit'].values[0]
        criteria['Sixth'].append(c6)

        check = set([c1, c2, c3, c4, c5, c6])
        if len(check) < 4:
            criteria['Enough Choices'].append(0)
        else:
            criteria['Enough Choices'].append(1)
        

    return criteria

In [78]:
for school in list(set(X.organization)):
    print(f'Working on: {school}')
    criteria = get_choices_by_school(school)
    df = pd.DataFrame(criteria)
    path = 'YS_Criteria_by_School/' + school + ' YSCriteria.csv'
    df.to_csv(path)
    print('Done.')

Working on: Rocky Fork Middle School
Done.
Working on: LaVergne Middle School
Done.
Working on: Thurman Francis Arts Academy
Done.
Working on: Oakland Middle School
Done.
Working on: Rock Springs Middle School
Done.
Working on: Smyrna Middle School
Done.
Working on: Blackman Middle School
Done.
Working on: Siegel Middle School
Done.
Working on: Christiana Middle School
Done.
Working on: Stewarts Creek Middle School
Done.
Working on: Rockvale Middle School
Done.
Working on: Whitworth-Buchanan Middle School
Done.


In [79]:
RCS_all = get_choices_by_school()
df = pd.DataFrame(RCS_all)
df.to_csv('YS_Criteria_by_School/All RCS YS Criteria.csv')

9309